<a href="https://colab.research.google.com/github/genaiconference/Agentic_KAG/blob/main/05_agentic_kag_inference_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic KAG


This notebook demonstrates the capabilities of Agentic KAG, a framework for building intelligent agents that can perform complex tasks and generate answers from Knoweldge Graphs.

In [1]:
!git clone https://github.com/genaiconference/Agentic_KAG.git

Cloning into 'Agentic_KAG'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 325 (delta 43), reused 30 (delta 30), pack-reused 268 (from 2)
Receiving objects: 100% (325/325), 7.20 MiB | 11.84 MiB/s, done.
Resolving deltas: 100% (166/166), done.


## Install Required Packages

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -r /content/Agentic_KAG/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.4 MB/s e

## Load credentials from .env

In [1]:
import os
os.chdir('/content/Agentic_KAG/')

from dotenv import load_dotenv

load_dotenv()  # This loads .env at project root

NEO4J_URI = os.getenv('AV_NEO4J_URL')
NEO4J_USERNAME = os.getenv('AV_NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('AV_NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set OPENAI_API_KEY as env variable for openai/neo4j-graphrag compatibility
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Initialize the Neo4j Driver
The Neo4j driver allows you to connect and perform read and write transactions with the database.

In [2]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# (Optional) Test the connection
driver.verify_connectivity()

## Initialize OpenAI LLM and Embeddings via neo4j-graphrag
We will use OpenAI **GPT-4.1**. The GraphRAG Python package supports any LLM model, including models from OpenAI, Google VertexAI, Anthropic, Cohere, Azure OpenAI, local Ollama models, and any chat model that works with LangChain. You can also implement a custom interface for any other LLM.

Likewise, we will use OpenAI’s **text-embedding-3-small** for the embedding model, but you can use other embedders from different providers.

In [3]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4.1",
    model_params={"temperature": 0}
)

embedder = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

We will also initialize langchain llm to use for community detection & Summarization

In [4]:
from langchain_openai import ChatOpenAI

# Initialize OpenAI LLM using LangChain
lang_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4.1", temperature=0)

Initialize OpenAI client

In [5]:
from openai import OpenAI

open_ai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## Initialize Langfuse Handler for Tracing

In [6]:
from langfuse.langchain import CallbackHandler
from langfuse import get_client

os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

langfuse_handler = CallbackHandler()

Langfuse client is authenticated and ready!


## Define ReACT Agent

In [7]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    PromptTemplate,
)
import re
import asyncio
from typing import List, Union
from langchain.agents import AgentOutputParser, AgentExecutor, create_react_agent
from langchain.schema import AgentAction, AgentFinish



# 1️⃣  ───── Robust multi-action parser ────────────────────────────
class MultiActionOutputParser(AgentOutputParser):
    """
    Extract *all* Action / Action Input pairs, even when inputs are
    multi-line or there is text between blocks.
    """
    _ACTION_RE = re.compile(
        r"""Action\s*\d*:\s*(.*?)\s*\n        # tool name
            Action\ Input\s*\d*:\s*([\s\S]*?) # tool input (non-greedy)
            (?=\nAction|\nObservation|\nFinal|\Z)  # stop at next block
        """,
        flags=re.IGNORECASE | re.VERBOSE,
    )

    def parse(self, text: str) -> Union[List[AgentAction], AgentFinish]:
        # ---------- collect all tool calls ----------
        matches = self._ACTION_RE.findall(text)
        if matches:
            return [
                AgentAction(
                    tool=tool.strip(),
                    tool_input=tool_input.strip().strip('"'),
                    log=text,
                )
                for tool, tool_input in matches
            ]

        # ---------- or a final answer ----------
        # final = re.search(r"(?:Final Answer|Final Thought):\s*(.*)", text, flags=re.IGNORECASE | re.DOTALL)
        final = re.search(r"(?:Final Answer):\s*(.*)", text, flags=re.IGNORECASE | re.DOTALL)

        # final = re.search(r"(?i)Final Answer:\s*(?:\*{2})?\s*\n+(.*)", text, flags=re.IGNORECASE | re.DOTALL)   ###PushToQA 04-07-2025 fixed ** issue

        if final:
            return AgentFinish(
                return_values={"output": final.group(1).strip()}, log=text
            )

        # ---------- fallback if no Action or Final Answer but appears like final output ----------
        if text.strip():  # fallback: any non-empty text is assumed to be the final answer
            return AgentFinish(
                return_values={"output": text.strip()}, log=text
            )

        raise ValueError(f"Could not parse agent output: {text!r}")


# 2️⃣  ───── Executor that really runs in parallel ────────────────
class ParallelAgentExecutor(AgentExecutor):
    async def _aiter_next_step(self, name_to_tool_map, inputs):
        llm_output = await self.agent.aplan(
            intermediate_steps=self.intermediate_steps, **inputs
        )
        parsed = self.output_parser.parse(llm_output)

        # ----- run *all* Actions concurrently -----
        if isinstance(parsed, list):
            coros = [self._aperform_agent_action(name_to_tool_map, a) for a in parsed]
            results = await asyncio.gather(*coros)

            for a, r in zip(parsed, results):
                self.intermediate_steps.append((a, r))
                yield f"Observation: {r}\n"

        elif isinstance(parsed, AgentFinish):
            self.intermediate_steps.append(
                (parsed, parsed.return_values["output"])
            )
            yield f"Final Answer: {parsed.return_values['output']}\n"


# 3️⃣  ───── Helper to build the agent + prompt ───────────────────
def get_react_agent(llm, tools, system_prompt, verbose=False):
    """Helper function for creating agent executor"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="conversation_history", optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        AIMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['agent_scratchpad'], template='{agent_scratchpad}')),
    ])
    agent = create_react_agent(llm, tools, prompt, output_parser=MultiActionOutputParser())

    executor = ParallelAgentExecutor(agent=agent,
                                     tools=tools,
                                     verbose=verbose,
                                     stream_runnable=True,
                                     handle_parsing_errors=True,
                                     max_iterations=20,
                                     return_intermediate_steps=True,
                                     )
    return executor

In [8]:
# from neo4j_graphrag.schema import get_schema
# get_schema(driver)

## Define Tools

Defining 4 tools used by the ReACT agent:

- **Hybrid Retrieval tool**
- **Hybrid Cypher Retrieval tool**
- **Global Search tool**
- **Local search tool**

In [9]:
from tools import av_hybrid_tool, av_hybrid_cypher_tool, global_retriever_tool, local_retriever_tool

Got 26 community summaries


## Agentic KAG Pipeline

In [10]:
from IPython.display import Markdown
import json
import prompts
from schema import manual_schema

def run_agentic_kag(query):
    """
    Runs the answer using react agent and provided tools.
    """
    # Run Hybrid Retrieval using all the available tools
    hybrid_agent = get_react_agent(
        lang_llm,
        [av_hybrid_tool, av_hybrid_cypher_tool, global_retriever_tool, local_retriever_tool],
        prompts.REACT_PROMPT,
        verbose=True
    )
    hybrid_input = json.dumps({
        "query": query,
    })
    answer = hybrid_agent.invoke({"input": hybrid_input,
                                  "SYSTEM_PROMPT" : prompts.AV_SYSTEM_PROMPT,
                                  }, config={"callbacks": [langfuse_handler]})
    final_answer = answer['output']
    return final_answer

In [11]:
from datetime import datetime
import pytz

def get_ist_datetime():
    """Return current IST date and time as separate strings."""
    ist = pytz.timezone("Asia/Kolkata")
    now_ist = datetime.now(ist)

    current_date = now_ist.strftime("%Y-%m-%d")
    current_time = now_ist.strftime("%H:%M")

    return current_date, current_time


# Example usage
def expand_question(question):
    date_str, time_str = get_ist_datetime()
    question = f"{question} Current Date: {date_str} Current Time: {time_str}"
    return question

## Let's test some questions

In [12]:
query = "What are the main topics covered in the first half of Day 1."
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...


Question: {"query": "What are the main topics covered in the first half of Day 1."}

Thought: To answer this, I need to identify all sessions and workshops scheduled for the first half of Day 1, then extract their main topics.
Action: AVHybridCypher(query: "List all sessions and workshops on Day 1 with their topics and start times. Filter for those in the first half of the day (morning to lunch).")

> Finished chain.


Question: {"query": "What are the main topics covered in the first half of Day 1."}

Thought: To answer this, I need to identify all sessions and workshops scheduled for the first half of Day 1, then extract their main topics.
Action: AVHybridCypher(query: "List all sessions and workshops on Day 1 with their topics and start times. Filter for those in the first half of the day (morning to lunch).")

In [13]:
query = "List down all the sessions or workshops which uses LangGraph?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "List down all the sessions or workshops which uses LangGraph?"}

Thought: This query requires structured search to identify all sessions or workshops that mention the use of LangGraph. I will use the AVHybridCypher tool to query the knowledge graph for this information.
Action: AVHybridCypher
Action Input: List all sessions and workshops where the technology or tool used includes 'LangGraph'.- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Type: Hack Session
  - Tools: LangGraph, FastMCP, LangMem, LangSmith

- **AutoGen vs CrewAI vs LangGraph: Battle of the Agent Frameworks**
  - Type: Hack Panel
  - Tools: AutoGen, CrewAI, LangGraph

- **Mastering Agentic Workflows with LangGraph**
  - Type: Hack Session
  - Tools: LangGraph

- **Mastering Intelligent Agents: A Deep Dive into Agentic AI**
  - Type: Workshop
  - Tools: LangChain, LangGraph, CrewAI, FastAPI, LangFuse, Arize AI Phoenix, Tavily, We

Final Thought: I have identified all sessions and workshops at DHS 2025 that use LangGraph as a tool or technology, based on the structured knowledge graph search.

Final Answer:

**Here are all the sessions and workshops at DHS 2025 that use LangGraph:**

- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Type: Hack Session
  - Tools: LangGraph, FastMCP, LangMem, LangSmith

- **AutoGen vs CrewAI vs LangGraph: Battle of the Agent Frameworks**
  - Type: Hack Panel
  - Tools: AutoGen, CrewAI, LangGraph

- **Mastering Agentic Workflows with LangGraph**
  - Type: Hack Session
  - Tools: LangGraph

- **Mastering Intelligent Agents: A Deep Dive into Agentic AI**
  - Type: Workshop
  - Tools: LangChain, LangGraph, CrewAI, FastAPI, LangFuse, Arize AI Phoenix, Tavily, WeatherAPI, Google Colab, Runpod.io, LangMem, Mem0

- **LLMOps – Productionalizing Real-World Applications with LLMs and Agents**
  - Type: Workshop
  - Tools: Hugging Face, Qdrant, Amazon SageMaker, LangGraph

**Here are all the sessions and workshops at DHS 2025 that use LangGraph:**

- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Type: Hack Session
  - Tools: LangGraph, FastMCP, LangMem, LangSmith

- **AutoGen vs CrewAI vs LangGraph: Battle of the Agent Frameworks**
  - Type: Hack Panel
  - Tools: AutoGen, CrewAI, LangGraph

- **Mastering Agentic Workflows with LangGraph**
  - Type: Hack Session
  - Tools: LangGraph

- **Mastering Intelligent Agents: A Deep Dive into Agentic AI**
  - Type: Workshop
  - Tools: LangChain, LangGraph, CrewAI, FastAPI, LangFuse, Arize AI Phoenix, Tavily, WeatherAPI, Google Colab, Runpod.io, LangMem, Mem0

- **LLMOps – Productionalizing Real-World Applications with LLMs and Agents**
  - Type: Workshop
  - Tools: Hugging Face, Qdrant, Amazon SageMaker, LangGraph, LangSmith, LangChain, AWS

- **AgentOps: Building and Deploying AI Agents**
  - Type: Workshop
  - Tools: LangChain, LangGraph

- **Building Intelligent Multimodal Agents: Integrating Vision, Speech & Language**
  - Type: Workshop
  - Tools: LangGraph, LangGraph Studio, SQLite, Qdrant, Groq LLMs, Whisper, ElevenLabs, Llama 3.2 Vision, Diffusion Models, Telegram Bot API, Telegram

- **Agentic RAG Workshop: From Fundamentals to Real-World Implementations**
  - Type: Workshop
  - Tools: Google Colab notebooks, Azure Document Intelligence, Chroma Vector DB, LangChain, LangGraph, Langfuse, Travily web search, Neo4j

---

If you found this helpful, do spread the word and recommend this app to your fellow attendees!  
Try it out at [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/) 🚀

In [14]:
query = "List down all the sessions or workshops which uses Agentic RAG?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...


Question: {"query": "List down all the sessions or workshops which uses Agentic RAG?"}

Thought: To answer this, I need to search for all sessions and workshops that mention "Agentic RAG" as a topic, technology, or methodology. This requires structured graph search to identify relevant sessions/workshops based on their descriptions or tags.
Action: AVHybridCypher(query: "Find all sessions and workshops where the topic, description, or technology includes 'Agentic RAG'")

> Finished chain.


Question: {"query": "List down all the sessions or workshops which uses Agentic RAG?"}

Thought: To answer this, I need to search for all sessions and workshops that mention "Agentic RAG" as a topic, technology, or methodology. This requires structured graph search to identify relevant sessions/workshops based on their descriptions or tags.
Action: AVHybridCypher(query: "Find all sessions and workshops where the topic, description, or technology includes 'Agentic RAG'")

In [ ]:
query = "I am a big lover of ai agents and I am interested in putting things in production. Suggest me a tailor made agenda, mention the name of the Session or workshop along with Instructor name?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Thought: The question asks for a customized agenda focused on AI agents and production deployment, including session/workshop names and instructor names. I need to search for relevant workshops or sessions, ideally with a practical/production focus, and list their details.
Action: GlobalRetrieval
Action Input: AI agent workshops and sessions for production deployment with instructor names

Processing communities in parallel: 100%|██████████| 26/26 [00:15<00:00,  1.72it/s]


# AI Agent Workshops and Sessions for Production Deployment (with Instructor Names)

A comprehensive review of the dataset reveals a rich set of workshops and sessions focused on the production deployment of AI agents, led by a diverse group of industry practitioners and experts. Below is a structured summary of the most relevant sessions, their focus areas, and the instructors involved.

---

## 1. End-to-End Agentic AI and LLMOps Workshops

- **Kartik Nighania (Typewise)**  
  Leads an end-to-end LLMOps workshop and hack session on engineering reliable agents at scale. Topics include RAG with Qdrant, SageMaker training/deployment, CI/CD, monitoring/logging, hallucination detection/guardrails, and memory systems.  
  [Data: Entities (15, 29, 435, 466, 16), Relationships (633, 1299, 3)]

- **Miguel Otero Pedrido (The Neural Maze)**  
  Conducts a multimodal agents workshop and hack session focused on moving beyond proofs of concept, with emphasis on agent monitoring, evaluation, simula

Thought: I now know the final answer.

Final Answer:

**Tailor-Made Agenda for AI Agents in Production**

---

**1. End-to-End Agentic AI and LLMOps**
- **Session:** Engineering Reliable Agents at Scale (LLMOps Workshop & Hack Session)
- **Instructor:** Kartik Nighania (Typewise)

---

**2. Multimodal Agent Systems**
- **Session:** Multimodal Agents: Beyond Proofs of Concept (Workshop & Hack Session)
- **Instructor:** Miguel Otero Pedrido (The Neural Maze)

---

**3. Mastering Real-World Agentic AI Applications**
- **Session:** Mastering Real-World Agentic AI Applications with AG2 (AutoGen)
- **Instructors:** Hrushikesh Dokala (Atlan; fAIght Club), Dr. Qingyun Wu (AG2; Penn State), Chi Wang (AG2; FLAML)

---

**4. AgentOps and Risk Mitigation**
- **Session:** Detecting and Mitigating Risks in Agentic AI
- **Instructor:** Bhaskarjit Sarmah (Domyn)

---

**5. Multi-Agent Frameworks and Workflows**
- **Session:** Building Multi-Agent Helpdesk AI Assistant (AutoGen, CrewAI, LangGraph Compa

**Tailor-Made Agenda for AI Agents in Production**

---

**1. End-to-End Agentic AI and LLMOps**
- **Session:** Engineering Reliable Agents at Scale (LLMOps Workshop & Hack Session)
- **Instructor:** Kartik Nighania (Typewise)

---

**2. Multimodal Agent Systems**
- **Session:** Multimodal Agents: Beyond Proofs of Concept (Workshop & Hack Session)
- **Instructor:** Miguel Otero Pedrido (The Neural Maze)

---

**3. Mastering Real-World Agentic AI Applications**
- **Session:** Mastering Real-World Agentic AI Applications with AG2 (AutoGen)
- **Instructors:** Hrushikesh Dokala (Atlan; fAIght Club), Dr. Qingyun Wu (AG2; Penn State), Chi Wang (AG2; FLAML)

---

**4. AgentOps and Risk Mitigation**
- **Session:** Detecting and Mitigating Risks in Agentic AI
- **Instructor:** Bhaskarjit Sarmah (Domyn)

---

**5. Multi-Agent Frameworks and Workflows**
- **Session:** Building Multi-Agent Helpdesk AI Assistant (AutoGen, CrewAI, LangGraph Comparison)
- **Instructors:** Praneeth Paikray (Manpower Group), Sanathraj Narayan (Data Science Manager)

---

**6. Agent Factory and Observability**
- **Session:** Agent Factory for Scaling Production Agents
- **Instructor:** Krishnakumar Menon

- **Session:** Architecting, Optimizing, and Observing Agentic Systems
- **Instructor:** Dipanjan Sarkar

- **Session:** Agent Intelligence Toolkit for Production
- **Instructor:** NVIDIA

---

**7. Interoperability and Protocols**
- **Session:** Interoperability with MCP and A2A Protocols
- **Instructors:** Nikhil Rana, Nitin Agarwal, Rutvik Acharya

---

**8. Robust, Scalable, and Multimodal Agent Systems**
- **Session:** Robust, Scalable Multi-Agent Systems for Production
- **Instructor:** Pranjal Singh

- **Session:** Modular, Fault-Tolerant SEO Content Pipeline
- **Instructor:** Anshu Kumar

- **Session:** Discovery and Novelty in LLM Outputs
- **Instructor:** Harshad Khadilkar

---

**9. Evaluation, Security, and Governance**
- **Session:** End-to-End LLM Evaluation for Production Deployment
- **Instructor:** Ayush Thakur (Weights & Biases, Google Developer Expert)

- **Session:** GenAI Model Evaluation and AI Red Teaming
- **Instructors:** Dr. Kiran R, Satnam Singh, Shivaraj Mulimani (Acalvio)

---

**10. Industry-Specific and Applied Sessions**
- **Session:** Building Agentic AI Systems for Aviation
- **Instructor:** Pradeep Kumar (Emirates Airlines)

- **Session:** Reliability in Agentic AI Systems (Human-in-the-Loop, Pydantic-AI)
- **Instructor:** Deepak Sharma

- **Session:** Serverless AI Workflow for Automotive Video Analysis
- **Instructor:** Vignesh Kumar

- **Session:** Scalable GenAI Deployment
- **Instructor:** Karan Sindwani (AWS)

---

**11. Healthcare and Customer Experience**
- **Session:** Scalable Healthcare Voice AI Contact Center
- **Instructors:** Ashish Tripathy (Pype AI), Mayank Aggarwal (evolvue AI/CreateHQ), Aashay Sachdeva (Sarvam AI)

- **Session:** Post-Training Alignment and Fine-Tuning of LLMs
- **Instructors:** Aashay Sachdeva, Vijay Gabale

- **Session:** AI-First Customer Experience Solutions
- **Instructors:** Pavak Biswal (DGS), Abhilash Kulkarni (Dentsu), Arijit Mallick (DGS)

---

**This agenda covers the full spectrum from agent design, deployment, monitoring, and evaluation to industry-specific applications, led by top practitioners and experts in the field** [source](#ai-agent-workshops-and-sessions-for-production-deployment-with-instructor-names).

In [ ]:
query = "I am a big lover of knowledge graphs (do not confuse with LangGraph). Suggest me a tailor made agenda, mention the name of the Session or workshop along with Instructor name and whn it is happening?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Thought: The question is about knowledge graphs (not LangGraph) and requests a tailored agenda with session/workshop names, instructor names, and timings. I need to search for relevant sessions or workshops about knowledge graphs, including their details.
Action: GlobalRetrieval
Action Input: knowledge graph sessions or workshops, instructor names, and schedule

Processing communities in parallel: 100%|██████████| 26/26 [00:14<00:00,  1.82it/s]


# Knowledge Graph Sessions or Workshops: Instructors and Schedule

## Overview

The dataset contains information on several sessions and workshops relevant to knowledge graphs and their applications, though not all are explicitly titled as "knowledge graph" sessions. Below is a synthesis of the available details, including session topics, instructor names, and scheduling information where provided.

---

## Explicit Knowledge Graph Session

- **Session Title:** The Missing Piece of AI Apps: Evaluation  
  **Type:** Day 2 Hack Session  
  **Date & Time:** 2025-08-21, 16:00–17:00  
  **Instructor:** Ayush Thakur (Manager, AI Engineer at Weights & Biases; Google Developer Expert in Machine Learning)  
  [Data: Entities (1285, 1286, 1287); Relationships (1226, 3297)]

---

## Related Workshops and Hack Sessions

### 1. LangGraph Workshop
- **Focus:** Graph-based agentic workflows, agent architectures with memory/tools, reflexion and error-handling loops, retrieval+reasoning, and experiment

Thought: I now know the final answer.

Final Answer:

**Tailor-Made Agenda for Knowledge Graph Enthusiasts**

---

**1. The Missing Piece of AI Apps: Evaluation**
- **Instructor:** Ayush Thakur (Manager, AI Engineer at Weights & Biases; Google Developer Expert in Machine Learning)
- **When:** 2025-08-21, 16:00–17:00
- **Type:** Day 2 Hack Session
- **Why Attend:** This session is explicitly focused on evaluation in AI apps, a crucial aspect of knowledge graph applications and their integration in AI systems.  
  [Source](#)

---

**2. Measuring Uncertainty in LLMs and Optimal Use of SLMs**
- **Instructor:** Kuldeep Jiwani (ConcertAI)
- **When:** Schedule not specified
- **Why Attend:** This session covers uncertainty in language models, which is highly relevant for knowledge graph-based reasoning and applications.  
  [Source](#)

---

**3. Fast and Accurate Conversational Agents: Beyond Function Calling**
- **Instructor:** Daksh Varshneya (Rasa)
- **When:** Schedule not specified
- **

**Tailor-Made Agenda for Knowledge Graph Enthusiasts**

---

**1. The Missing Piece of AI Apps: Evaluation**
- **Instructor:** Ayush Thakur (Manager, AI Engineer at Weights & Biases; Google Developer Expert in Machine Learning)
- **When:** 2025-08-21, 16:00–17:00
- **Type:** Day 2 Hack Session
- **Why Attend:** This session is explicitly focused on evaluation in AI apps, a crucial aspect of knowledge graph applications and their integration in AI systems.  
  [Source](#)

---

**2. Measuring Uncertainty in LLMs and Optimal Use of SLMs**
- **Instructor:** Kuldeep Jiwani (ConcertAI)
- **When:** Schedule not specified
- **Why Attend:** This session covers uncertainty in language models, which is highly relevant for knowledge graph-based reasoning and applications.  
  [Source](#)

---

**3. Fast and Accurate Conversational Agents: Beyond Function Calling**
- **Instructor:** Daksh Varshneya (Rasa)
- **When:** Schedule not specified
- **Why Attend:** Conversational agents often leverage knowledge graphs for context and reasoning, making this session valuable for practical applications.  
  [Source](#)

---

**4. From Idea to Production with GenAI**
- **Instructor:** Karan Sindwani (AWS Senior Applied Scientist)
- **When:** Day 2
- **Why Attend:** Focuses on productionizing GenAI, which often involves knowledge graph integration for real-world deployments.  
  [Source](#)

---

**5. Mastering Real-World Agentic AI Applications with AG2 (AutoGen)**
- **Instructors:** Hrushikesh Dokala (Atlan; fAIght Club), Dr. Qingyun Wu (AG2; Penn State), Chi Wang (AG2; FLAML)
- **When:** Schedule not specified
- **Why Attend:** This workshop covers agentic AI applications, which frequently utilize knowledge graphs for memory and reasoning.  
  [Source](#)

---

**Summary Table**

| Session Title / Topic                                      | Instructor(s)                                   | Date / Schedule           |
|------------------------------------------------------------|-------------------------------------------------|---------------------------|
| The Missing Piece of AI Apps: Evaluation                   | Ayush Thakur                                    | 2025-08-21, 16:00–17:00   |
| Measuring Uncertainty in LLMs and Optimal Use of SLMs      | Kuldeep Jiwani                                  | Not specified             |
| Fast and Accurate Conversational Agents                    | Daksh Varshneya                                 | Not specified             |
| From Idea to Production with GenAI                         | Karan Sindwani                                  | Day 2                     |
| Mastering Real-World Agentic AI Applications with AG2      | Hrushikesh Dokala, Dr. Qingyun Wu, Chi Wang     | Not specified             |

---

**Note:** Only the first session has a precise date and time. For the others, please check the event’s official schedule for updates.  
[Source: GlobalRetrieval results]

In [ ]:
query = " Which specific company has the most speakers participating in DHS 2025?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Thought: To answer this, I need to find a list of speakers for DHS 2025 and identify their company affiliations, then determine which company has the highest number of speakers.
Action: GlobalRetrieval
Action Input: List of speakers and their company affiliations for DHS 2025

Processing communities in parallel: 100%|██████████| 26/26 [00:08<00:00,  2.93it/s]


# List of Speakers and Company Affiliations for DHS 2025

Below is a consolidated list of speakers for DHS 2025, along with their company affiliations where available. In several cases, the company affiliation is not specified in the provided data.

---

## Confirmed Speakers with Company Affiliations

- **Syed Quiser Ahmed** — Infosys  
- **Kartik Nighania** — Typewise  
- **Miguel Otero Pedrido** — The Neural Maze  
- **Raghav Bali** — Delivery Hero  
- **Hrushikesh Dokala** — Atlan, fAIght Club  
- **Dr. Qingyun Wu** — AG2, Penn State  
- **Chi Wang** — AG2, FLAML  
- **Deepak Sharma** — Google/DeepMind  
- **Luis Serrano** — Serrano Academy (prior roles at Google, Apple, Cohere, Zapata)  
- **Bhaskarjit Sarmah** — Domyn  
- **Praneeth Paikray** — Manpower Group (prior: TCS, Dell, Fidelity)  
- **Kuldeep Jiwani** — ConcertAI  
- **Daksh Varshneya** — Rasa  
- **Pradeep Kumar** — Emirates Airlines  
- **Vignesh Kumar** — Google Cloud  
- **Pavak Biswal** — Dentsu Global Services (DGS

Thought: I need to count the number of speakers from each company based on the provided affiliations to determine which company has the most speakers.

- Dentsu/Dentsu Global Services (DGS): Pavak Biswal, Abhilash Kulkarni, Arijit Mallick (3 speakers)
- Google/Google Cloud/DeepMind: Deepak Sharma, Vignesh Kumar (2 speakers; Luis Serrano is ex-Google, not current)
- Acalvio: Satnam Singh, Shivaraj Mulimani (2 speakers)
- AG2: Dr. Qingyun Wu, Chi Wang (2 speakers)
- Atlan/fAIght Club: Hrushikesh Dokala (1 speaker)
- Infosys: Syed Quiser Ahmed (1 speaker)
- Typewise: Kartik Nighania (1 speaker)
- The Neural Maze: Miguel Otero Pedrido (1 speaker)
- Delivery Hero: Raghav Bali (1 speaker)
- Domyn: Bhaskarjit Sarmah (1 speaker)
- Manpower Group: Praneeth Paikray (1 speaker)
- ConcertAI: Kuldeep Jiwani (1 speaker)
- Rasa: Daksh Varshneya (1 speaker)
- Emirates Airlines: Pradeep Kumar (1 speaker)
- Samsung Electronics: Praveen Kumar GS (1 speaker)
- Weights & Biases: Ayush Thakur (1 speaker)
- 

- **Dentsu/Dentsu Global Services (DGS)** has the most speakers participating in DHS 2025, with three confirmed speakers: Pavak Biswal, Abhilash Kulkarni, and Arijit Mallick [Speaker list and affiliations, see above].

In [ ]:
query = " Tell me one unique session in the conference?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Thought: To answer this, I need to identify a session in the conference that stands out as unique. I will search for sessions and look for one with distinctive features.
Action: GlobalRetrieval
Action Input: unique session in the conference

Processing communities in parallel: 100%|██████████| 26/26 [00:13<00:00,  1.98it/s]


# Unique Sessions in the Conference

The conference features a diverse array of unique sessions, each distinguished by its focus, format, or technical depth. Below is a synthesis of the most notable and distinctive sessions, highlighting their key characteristics and implications.

---

## 1. **MassGen: Live Multi-Agent Collaboration**
A standout session is **MassGen**, which orchestrates a live, real-time collaboration between multiple large language models—Claude, GPT, and Grok. This session is unique in its demonstration of cross-model synergy and parallel workflows, showcasing the combined capabilities of different LLMs in a single, interactive environment. The focus on real-time, multi-agent interaction sets it apart from traditional single-model demonstrations [Data: Entities (370, 847, 1313, 1549, 2591); Relationships (900, 1998, 3756, 1221, 2004)].

---

## 2. **Hands-On Technical Workshops**
Several workshops offer hands-on, production-grade training that distinguishes them fr

Thought: I now know the final answer.
Final Answer: 

**One Unique Session in the Conference**

- **MassGen: Live Multi-Agent Collaboration**
  - This session is unique because it features a live, real-time collaboration between multiple large language models—Claude, GPT, and Grok.
  - It demonstrates cross-model synergy and parallel workflows, showcasing the combined capabilities of different LLMs in a single, interactive environment.
  - The focus on real-time, multi-agent interaction sets it apart from traditional single-model demonstrations.  
  - [Source: Conference Unique Sessions Overview](#unique-sessions-in-the-conference)

> Finished chain.


**One Unique Session in the Conference**

- **MassGen: Live Multi-Agent Collaboration**
  - This session is unique because it features a live, real-time collaboration between multiple large language models—Claude, GPT, and Grok.
  - It demonstrates cross-model synergy and parallel workflows, showcasing the combined capabilities of different LLMs in a single, interactive environment.
  - The focus on real-time, multi-agent interaction sets it apart from traditional single-model demonstrations.  
  - [Source: Conference Unique Sessions Overview](#unique-sessions-in-the-conference)

# Run all retrievers

In [8]:
import pandas as pd
from tqdm import tqdm
from IPython.display import Markdown, display
from tools import (
    get_rag_for_query_text2cypher,
    get_rag_for_query_hybrid,
    get_rag_for_query_hybrid_cypher,
    global_retriever,
    local_search,
    community_data,   # required for global_retriever
)

# Your list of questions
complex_questions = [
    "Which company has the most speakers participating in DHS 2025?",
    "How many Microsoft employees are speaking at DHS 2025, and who are they?",
    "Identify all sessions where all speakers are from the same organization – name the session and the company",
    "Name a speaker at DHS 2025 who is involved in more than one session, and list the sessions they are presenting",
    "What sessions at DHS 2025 have multiple speakers? Provide the session titles along with the speaker names",
    "Which three companies have the highest number of speakers at DHS 2025?",
    "Which workshop at DHS 2025 is specifically designed for business leaders, and who is the instructor?",
    "If you’re interested in “AgentOps”, which session should you attend and who is leading that session?",
    "List all sessions that explicitly mention “Responsible AI” in their title or theme",
    "Name two sessions at DHS 2025 that focus on multimodal AI applications",
    "How many speakers at DHS 2025 hold a title of “Founder” or “Co-Founder”, and can you give two examples of such speakers?",
    "Which speakers are from NeuralHQ.ai, and which session do they co-present together?",
    "Joshua Starmer and Luis Serrano are both speaking at DHS 2025. How many sessions is each of them involved in, and what are those sessions?",
    "What are the titles and who are the speakers of all the Power Talk sessions at DHS 2025?",
    "If someone wants to learn about LLMs at DHS 2025, which sessions or workshops should they attend (name at least two)?",
    "Which session is co-presented by an employee from Toast and an employee from Atlassian, and what topic does it cover?",
    "Provide an example of a session title that is phrased as a question, and name the speaker who is presenting it",
    "Name two sessions that deal with AI safety or incorporate “guardrails” in AI, and identify their speakers",
    "Who are the Pure Storage speakers at DHS 2025, and what topics are they presenting?",
    "Identify sessions that involve AI agent frameworks (such as LangGraph or AG2) and name the speakers leading those sessions",]

theme_based_questions = ["Which topics are the most frequently covered across all sessions and workshops at DHS 2025?",
    "What are the top 3 tools used across both workshops and sessions at this year’s conference?",
    "Which theme connects the largest number of sessions at DHS 2025?",
    "How many sessions cover LLMs and how many cover agents? Is there any session that covers both?",
    "Which speakers at DHS 2025 are associated with multiple topics across different formats (session + workshop)?",
    "What are the most common prerequisites across all workshops this year?",
    "Identify the clusters of speakers who are part of the same company and covering similar or connected topics",
    "Which companies are most active in DHS 2025 — across speaking, sponsoring, and conducting workshops?",
    "What are the top 5 skills or tools attendees will be exposed to if they attend sessions on the Enterprise GenAI theme?",
    "What are the emerging subfields of GenAI based on the diversity of topics this year?",
    "How many sessions explicitly deal with ethics, safety, or responsible AI, and who’s leading that conversation?",
    "What’s the distribution of session difficulty levels (beginner/intermediate/advanced) across the full agenda?",
    "Which themes are most commonly associated with Power Talks versus Hack Sessions?",
    "Who are the influential nodes in the speaker network — people connected to many topics, sessions, and organizations?",
    "Which tools appear only in sessions but not in any workshop, and what might that suggest?",
    "Identify multi-hop knowledge paths between Arun Prakash Asokan’s session and other speakers working on similar tools or topics",
    "What are the top 5 most interconnected people in the conference graph, based on session, workshop, and organization links?",
    "Which organizations have both sponsored and presented sessions or workshops, and what topics do they represent?",
    "If someone wants to follow only workshops that cover Agentic AI, which ones should they attend and what prerequisites do they need?",
    "What are the gaps in this year’s conference — for example, highly-discussed topics that have no workshop coverage?"]

simple_questions = ["On what dates and at which venue is DataHack Summit 2025 being held?",
    "What is the theme (tagline) of DHS 2025?",
    "Approximately how many attendees are expected to join DHS 2025 this year?",
    "Roughly how many AI talks are expected at the conference?",
    "About how many hack sessions are planned for DHS 2025?",
    "Name two types of session formats that DHS 2025 will feature (e.g., Keynote, Workshop, etc.)",
    "Who is delivering the keynote session “Responsible AI in Medical Imaging – A Case Study”?",
    "Who is Arun? List down all the sessions or workshops delivered by him",
    "Which founder of StatQuest is speaking at the conference?",
    "Who is the founder of Serrano Academy that is featured as a speaker at DHS 2025?",
    "Name one workshop being offered at DHS 2025 (give the title of the workshop)",
    "Which workshop at DHS 2025 is led by Dipanjan Sarkar?",
    "Which workshop covers productionalizing real-world applications with LLMs and agents, and who is the instructor?",
    "Who is giving the Power Talk titled “Revolutionizing AI for Relational Databases: No More Flattening”?",
    "Name a speaker from NVIDIA who is speaking at DHS 2025",
    "Is there a Hugging Face engineer speaking at DHS 2025? If yes, who is it?",
    "Name a speaker from Pure Storage who is speaking at DHS 2025",
    "Who is presenting the hack session “Quantifying Our Confidence in Neural Networks and AI”?",
    "What is the title of Pankaj Agarwal’s hack session at DHS 2025?",
    "What Power Talk is Syed Quiser Ahmed (Infosys) delivering at the summit?"]


medium_questions= [ "What are the three main focuses of the “AI Trinity” theme at DHS 2025?",
    "Which DHS 2025 session is co-presented by two speakers from Acalvio Technologies, and what is it called?",
    "Who are the two Microsoft speakers leading the session on AI Voice Agents, and what are their names?",
    "Which co-founder of AG2 is conducting a workshop at DHS 2025?",
    "Name one session title at DHS 2025 that includes “Retrieval-Augmented Generation (RAG)” in its title",
    "What is the title of the hack session presented by Analytics Vidhya’s Gyan Prakash Tripathi?",
    "Who is speaking about building a healthcare voice AI contact center (Pipecat) at DHS 2025, and what is the title of that session?",
    "Who is delivering the Power Talk on evaluating GenAI models in enterprise and healthcare, and which organization is he from?",
    "Which DHS 2025 session covers aligning Responsible AI with LLMs & Agents, and who is the speaker for it?",
    "Is there a session on AI agent interoperability protocols (e.g., A2A, MCP) at the conference? If yes, what is its title?",
    "Who is presenting on building real-world agentic AI systems beyond PoCs, and what is the speaker’s name?",
    "What session at DHS 2025 focuses on using AI for SEO content generation, and what is it called?",
    "Who is the Microsoft Generative AI leader speaking at DHS 2025 (name the person)?",
    "Who is giving the talk titled “Measuring Uncertainty in LLMs and Optimal Use of SLMs”, and what is the speaker’s name?",
    "Who is discussing multi-agent AI for public travel systems, and which company does that speaker represent?",
    "Which speaker from Pure Storage is talking about deploying GenAI safely, and what is the title of her talk?",
    "Give an example of a session title at DHS 2025 that includes the term “Agentic AI” in it",
    "Who is the Founder & CEO of Analytics Vidhya that is speaking at DHS 2025?",
    "Name one Oracle speaker who is delivering a Power Talk at DHS 2025",
    "Joshua Starmer and Luis Serrano are jointly leading which workshop at DHS 2025?",
]

example_questions = [
    # 1) BASIC RETRIEVAL
    "List all speakers at the conference.",
    "Display all sessions.",
    "Show all workshops.",
    "List all tools.",
    "List all topics.",
    "List all venues.",
    "Show all sponsors.",
    "List all companies represented.",
    "List conferences by year.",
    "List all educational institutions speakers studied at.",

    # 2) DIRECT RELATIONSHIPS
    "Who presents each session?",
    "Who conducted each workshop?",
    "Which sessions cover the topic LLMs?",
    "Which workshops cover Agentic RAG?",
    "What tools are used in the session titled 'LLM Scalability Challenges'?",
    "What sessions use LangChain?",
    "Which workshops use PyTorch?",
    "Which companies employ speakers attending the conference?",
    "Show sponsors of DataHack Summit 2025.",
    "What content is hosted by the conference (sessions & workshops)?",
    "What is the venue for each session?",
    "What is the venue for each workshop?",
    "List hack panel speakers.",
    "Which speakers present Hack Panel sessions?",
    "List speakers and their alma maters.",
    "Which venue hosts the conference?",

    # 3) FILTERING & CONDITIONS
    "List all intermediate-level workshops.",
    "Find sessions on Day 2.",
    "Show sessions starting after 10:00 AM on Day 1.",
    "Show workshops starting after 10:00.",
    "Sessions between 14:00 and 15:30.",
    "Sessions that use both LangChain and Neo4j.",
    "Sessions covering RAG or Agentic RAG.",
    "Speakers with RAG-related skills.",
    "Speakers who have awards listed.",
    "Speakers certified in AWS or Azure.",
    "Venues hosting more than 3 sessions.",
    "Topics with at least 5 total sessions/workshops.",
    "Most used tools across the program.",
    "Companies with at least two speakers.",
    "Sponsor counts per conference.",

    # 4) AGGREGATIONS & STATS
    "Count the number of sessions by theme.",
    "How many workshops did each speaker conduct?",
    "How many sessions did each speaker present?",
    "How many hack panel speakers are there?",
    "Number of sessions per day per venue.",
    "Top 5 topics by coverage across sessions and workshops.",
    "For each speaker, how many distinct tools are used in their sessions?",
    "For each company, how many presenting speakers?",
    "Average number of sessions per venue.",
    "Session count by session type.",

    # 5) PATTERN-BASED & RECOMMENDATIONS
    "Recommend workshops for someone interested in LLMs and LangChain.",
    "Workshops using the same tools as the session 'Graph at Scale'.",
    "Sessions similar to 'Graph at Scale' by overlapping topics and tools.",
    "Recommend sessions based on a list of interests (topics).",
    "Find sessions that do not use any tools.",
    "Find sessions that cover both RAG and Vector Databases.",
    "Find sessions covering any of the given topics.",
    "Who co-presented with a given speaker?",
    "Sessions presented by speakers from a specific company.",
    "Beginner workshops covering a given topic.",
    "Sessions at the Claude venue on Day 1 sorted by time.",
    "Sessions that use tools used by workshops covering Agentic RAG.",

    # 6) ADVANCED / SUPER-COMPLEX
    "Detect time overlaps between sessions in the same venue (same day).",
    "Find gaps between consecutive sessions per venue and day.",
    "Shortest collaboration path between two speakers (up to 6 hops).",
    "Sessions whose topic and tool sets are supersets of a reference session.",
    "Return rich session objects with venue, tools, and topics.",
    "Unified content using tool Neo4j (sessions & workshops).",
    "Speakers with no company listed.",
    "Find sessions instructed by Jane Doe (property search).",
    "Speakers who studied at any of a given list of universities.",
    "For each topic, list all speakers involved via sessions or workshops.",
    "Find venues that host both sessions and workshops.",
    "Which sponsors support more than one conference?",
    "Detect scheduling conflicts for a specific speaker’s sessions.",
    "Speakers whose sessions use a given tool.",
    "Pairs of speakers who studied at the same institution.",
    "Case-insensitive search for topics by substring.",
    "Sessions and workshops that cover RAG, including conference context.",
    "Build a venue-wise session schedule map."
]


Got 26 community summaries


In [ ]:
results = []

# List of question types to iterate through
question_types = {
    #"simple_questions": simple_questions,
    #"medium_questions": medium_questions,
    #"complex_questions": complex_questions,
    #"theme_based_questions": theme_based_questions,
    "example_questions": example_questions,
}

for type_name, questions_list in question_types.items():
    print(f"Running retrievers for {type_name}...")
    results = [] # Reset results for each question type

    for q in tqdm(questions_list, desc=f"Running retrievers for {type_name}"):
        # Print the current question in bold Markdown for clarity
        display(Markdown(f"**Running question:** {q}"))

        try:
            text2cypher_answer = get_rag_for_query_text2cypher(q)
            print(text2cypher_answer)
        except Exception as e:
            text2cypher_answer = f"Error: {e}"

        try:
            hybrid_answer = get_rag_for_query_hybrid(q)
            print(hybrid_answer)
        except Exception as e:
            hybrid_answer = f"Error: {e}"

        try:
            hybrid_cypher_answer = get_rag_for_query_hybrid_cypher(q)
            print(hybrid_cypher_answer)
        except Exception as e:
            hybrid_cypher_answer = f"Error: {e}"

        try:
            global_search_answer = global_retriever(q, community_data)
        except Exception as e:
            global_search_answer = f"Error: {e}"

        try:
            local_search_answer = local_search(q)
        except Exception as e:
            local_search_answer = f"Error: {e}"

        # Append row with Markdown answers as raw text
        results.append({
            "question": q,
            "text2cypher_answer": str(text2cypher_answer),
            "hybrid_answer": str(hybrid_answer),
            "hybrid_cypher_answer": str(hybrid_cypher_answer),
            "global_search_answer": str(global_search_answer),
            "local_search_answer": str(local_search_answer),
        })

    # Convert results into DataFrame
    df = pd.DataFrame(results)

    # Save to Excel
    output_file = f"{type_name}_results.xlsx"
    df.to_excel("/content/drive/MyDrive/" +output_file, index=False)

    print(f"✅ Results for {type_name} saved to {output_file}")

Running retrievers for example_questions...


Running retrievers for example_questions:   0%|          | 0/81 [00:00<?, ?it/s]

**Running question:** List all speakers at the conference.

- **Mohammad Sanad Zaki Rizvi**
  - [LinkedIn Profile](https://www.linkedin.com/in/mohd-sanad-zaki-rizvi-0238b5a6/)

*No other speaker names are provided in the context.*
<module 'cypher' from '/content/Agentic_KAG/cypher.py'>
- **Shivaraj Mulimani** – [LinkedIn](https://www.linkedin.com/in/shivaraj-mulimani-3445b0a9/)
- **Satnam Singh** – [LinkedIn](https://www.linkedin.com/in/satnamsinghdatascientist/)
- **Vignesh Kumar** – [LinkedIn](https://www.linkedin.com/in/vignesh-kumar-56555a94/)
- **Tanika Gupta** – [LinkedIn](https://www.linkedin.com/in/tanika-gupta-78242423/)
- **Rutvik Acharya** – [LinkedIn](https://www.linkedin.com/in/rutvikacharya/)
- **Nitin Agarwal** – [LinkedIn](https://www.linkedin.com/in/agnitin/)
- **Manpreet Singh** – [LinkedIn](https://www.linkedin.com/in/singhmanpreet2517/)
- **Syed Quiser Ahmed** – [LinkedIn](https://www.linkedin.com/in/syedquiserahmed/)
- **Daksh Varshneya** – [LinkedIn](https://www.linkedin.com/in/dakshvar/)
- **Arun Prakash Asokan** – [Linke


Running retrievers for example_questions:   1%|          | 1/81 [01:12<1:36:16, 72.20s/it]

...

**Running question:** List all tools.

- **Write-Select-Compress-Isolate framework**
- **Agent architectures with memory and tools**
- **RAHAT (Responsive AI Helper and Tasker)**
<module 'cypher' from '/content/Agentic_KAG/cypher.py'>
- **Garak**
- **PyRIT**
- **ART**



Running retrievers for example_questions:   5%|▍         | 4/81 [07:37<2:28:58, 116.08s/it]

**Running question:** List all topics.

- **The Neural Maze**  
- **Dr. Geetha Manjunath**  
- **Research, summarization, and decision-making**  
- **Visualizations from natural language queries**  
- **Write-Select-Compress-Isolate framework**
<module 'cypher' from '/content/Agentic_KAG/cypher.py'>


In [1]:
len(results)

NameError: name 'results' is not defined